# Drink Import Program
## Loads data from www.thecocktaildb.com and converts it to CSV.

In [7]:
import json
import csv
import urllib.request
import unidecode
from pathlib import Path

### File Locations

In [8]:
save_location = "rawdata"

### API Functions

In [9]:
def get_drink_dict(apicall):
    with urllib.request.urlopen(apicall) as url:
        my_dict = json.loads(url.read().decode())
        print(my_dict['drinks'])
        return my_dict;


def create_csv_simple(apicall, csvname):
    my_dict = get_drink_dict(apicall)
    outpath = Path.cwd() / save_location / csvname
    with open(outpath, 'w', newline='', encoding='utf-8') as f:
        w = csv.DictWriter(f, my_dict['drinks'][0].keys())
        w.writeheader()
        w.writerows(my_dict['drinks'])
        return my_dict;


def get_all_recipes(apicall, csvname, mydict):
    outpath = Path.cwd() / save_location / csvname
    with open(outpath, 'w', newline='', encoding='utf-8') as f:

        with urllib.request.urlopen(apicall + str(mydict['drinks'][0]['idDrink'])) as url:
            single_drink = json.loads(url.read().decode())
        w = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        w.writerow(single_drink['drinks'][0].keys())

        for drink in mydict['drinks']:
            with urllib.request.urlopen(apicall + str(drink['idDrink'])) as url:
                single_drink = json.loads(url.read().decode())
                print(single_drink['drinks'])
                w.writerow(single_drink['drinks'][0].values());

### Data Import
## THIS TAKES A LONG TIME TO RUN! DO NOT RUN UNLESS MODIFYING OR REBUILDING FROM SCRATCH!
The following cell will download all of the data from www.thecocktaildb.com one recipe at a time. The API only allows for full recipe details on a single drink at a time.

In [12]:
%%capture
# Load ingredient list
apicall = "https://www.thecocktaildb.com/api/json/v2/9973533/list.php?i=list"
csvname = 'ingredients.csv'
create_csv_simple(apicall, csvname);

# Load popular drinks
apicall = "https://www.thecocktaildb.com/api/json/v2/9973533/popular.php"
csvname = 'popular.csv'
create_csv_simple(apicall, csvname);

# Load recently added drinks
apicall = "https://www.thecocktaildb.com/api/json/v2/9973533/recent.php"
csvname = 'recent.csv'
create_csv_simple(apicall, csvname);

# Load alcoholic drinks
apicall = "https://www.thecocktaildb.com/api/json/v2/9973533/filter.php?a=Alcoholic"
csvname = 'alcoholic.csv'
my_dict = create_csv_simple(apicall, csvname);  # Save dictionary for use on full list of recipes

# Build the large table of all alcoholic recipes
apicall = "https://www.thecocktaildb.com/api/json/v2/9973533/lookup.php?i="
csvname = 'alcoholic_full.csv'
get_all_recipes(apicall, csvname, my_dict);

# Load non-alcoholic drinks
apicall = "https://www.thecocktaildb.com/api/json/v2/9973533/filter.php?a=Non_Alcoholic"
csvname = 'non-alcoholic.csv'
my_dict = create_csv_simple(apicall, csvname);  # Save dictionary for use on full list of recipes

# Build the large table of all non-alcoholic recipes
apicall = "https://www.thecocktaildb.com/api/json/v2/9973533/lookup.php?i="
csvname = 'non-alcoholic_full.csv'
get_all_recipes(apicall, csvname, my_dict);
